# MNIST 
### **Feedforward Fully Connected SNN**

This tutorial goes over how to train a simple feedforward SNN and deploy on HiAER Spike using our 
conversion pipline.

### **Define a Feedforward SNN**
To build a simple feedforward spiking neural network with PyTorch, we can use snnTorch, SpikingJelly or other deep learning frameworks that are based on PyTorch. Currently, our conversion pipline supports snnTorch and SpikingJelly. In this tutorial, we will be using SpikingJelly.

Install the PyPi distribution of SpikingJelly

In [1]:
pip install spikingjelly


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries from SpikingJelly and PyTorch

In [2]:
from spikingjelly.activation_based import neuron, functional, surrogate
import torch 
import torch.nn as nn

### **Model Architecture**
Using SpikingJelly, we can define a simple 2-layer feedforward SNN model with 1000 hidden neurons. The PyTorch layer will act as synapses between the spiking neuron layers. 
#### **Surrogate Function**
SpikingJelly and snnTorch both use backpropagation through time to train the spiking neural networks. However, because of the non-differentiability of spikes, surrogate gradients are used in place of the Heaviside function in the backward pass

In [3]:
class model(nn.Module): 
    def __init__(self, features = 1000): 
        super().__init__() 
        self.flat = nn.Flatten()
        self.linear1 = nn.Linear(28 * 28, features, bias=False) 
        self.lif1 = neuron.LIFNode(surrogate_function=surrogate.ATan()) 
        self.linear2 = nn.Linear(features, 10, bias=False) 
        self.lif2 = neuron.LIFNode(surrogate_function=surrogate.ATan()) 
    def forward(self, x): 
        x = self.flat(x)
        x = self.linear1(x) 
        x = self.lif1(x) 
        x = self.linear2(x) 
        x = self.lif2(x) 
        return x

In [4]:
#Initiate the Network
net = model()

### **Setting up the MNIST Dataset**

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Download MNIST data from torch 
mnist_train = datasets.MNIST('data/mnist', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))
mnist_test = datasets.MNIST('data/mnist', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))

# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=128, shuffle=True, drop_last=True)


### **Training the SNN**
Since we are using a static image dataset, we will first encode the image into spikes using the rate encoding function from spikingjelly. With rate encoding, the input feature determines the firing frequency and the neuron that fries the most is selected as the predicted class.  

In [6]:
from spikingjelly.activation_based import encoding
import time

In [7]:
#Setting up the encoder and the time steps
encoder = encoding.PoissonEncoder()
num_steps = 20

#Define training parameters
epochs = 10
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#Copy netowrk to device 
net.to(device)

#Define optimizer, scheduler and the loss function
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
loss_fun = torch.nn.MSELoss()

In [8]:
for epoch in range(epochs):
    start_time = time.time()
    net.train()
    train_loss = 0
    train_acc = 0
    train_samples = 0
    for img, label in train_loader:
        optimizer.zero_grad()
        img = img.to(device)
        label = label.to(device)
        label_onehot = torch.nn.functional.one_hot(label, 10).float()
        out_fr = 0.
        for t in range(num_steps):
            encoded_img = encoder(img)
            out_fr += net(encoded_img)
        out_fr = out_fr/num_steps  
        loss = loss_fun(out_fr, label_onehot)
        loss.backward()
        optimizer.step()

        train_samples += label.numel()
        train_loss += loss.item() * label.numel()
        train_acc += (out_fr.argmax(1) == label).float().sum().item()

        #reset the membrane protential after each input image
        functional.reset_net(net)

    train_time = time.time()
    train_speed = train_samples / (train_time - start_time)
    train_loss /= train_samples
    train_acc /= train_samples
    
    # lr_scheduler.step()
        
    net.eval()
    test_loss = 0
    test_acc = 0
    test_samples = 0

    with torch.no_grad():
        for img, label in test_loader:
            img = img.to(device)
            label = label.to(device)
            label_onehot = torch.nn.functional.one_hot(label, 10).float()
            out_fr = 0.   
            for t in range(num_steps):
                encoded_img = encoder(img)
                out_fr += net(encoded_img)
            out_fr = out_fr/num_steps  

            loss = loss_fun(out_fr, label_onehot)

            test_samples += label.numel()
            test_loss += loss.item() * label.numel()
            test_acc += (out_fr.argmax(1) == label).float().sum().item()
            functional.reset_net(net)

    test_time = time.time()
    test_speed = test_samples / (test_time - train_time)
    test_loss /= test_samples
    test_acc /= test_samples

    print(f'epoch = {epoch}, train_loss ={train_loss: .4f}, train_acc ={train_acc: .4f}, test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}')
    print(f'train speed ={train_speed: .4f} images/s, test speed ={test_speed: .4f} images/s')

epoch = 0, train_loss = 0.0190, train_acc = 0.8882, test_loss = 0.0083, test_acc = 0.9552
train speed = 4073.1702 images/s, test speed = 6908.0617 images/s
epoch = 1, train_loss = 0.0066, train_acc = 0.9659, test_loss = 0.0054, test_acc = 0.9712
train speed = 4541.4101 images/s, test speed = 10556.4562 images/s
epoch = 2, train_loss = 0.0045, train_acc = 0.9778, test_loss = 0.0046, test_acc = 0.9744
train speed = 4592.4925 images/s, test speed = 10618.9597 images/s
epoch = 3, train_loss = 0.0034, train_acc = 0.9838, test_loss = 0.0042, test_acc = 0.9777
train speed = 4585.2458 images/s, test speed = 10365.1217 images/s
epoch = 4, train_loss = 0.0027, train_acc = 0.9882, test_loss = 0.0037, test_acc = 0.9814
train speed = 4540.4732 images/s, test speed = 10695.3007 images/s
epoch = 5, train_loss = 0.0022, train_acc = 0.9914, test_loss = 0.0033, test_acc = 0.9825
train speed = 4541.5212 images/s, test speed = 10449.7839 images/s
epoch = 6, train_loss = 0.0018, train_acc = 0.9932, test_lo

### **Converting the trained SNN to HiAER Spike Format**

In [9]:
from hs_api.converter.cri_converter import CRI_Converter, Quantize_Network
from hs_api.api import CRI_network
import hs_bridge

#Weight, Bias Quantization 
qn = Quantize_Network() 
net_quan = qn.quantize(net)

#Set the parameters for conversion
input_layer = 1 #first pytorch layer that acts as synapses
output_layer = 4 #last pytorch layer that acts as synapses
input_shape = (1, 28, 28)
backend = 'spikingjelly'
v_threshold = qn.v_threshold

cn = CRI_Converter(num_steps = num_steps, 
                   input_layer = input_layer, 
                   output_layer = output_layer, 
                   input_shape = input_shape,
                   backend=backend,
                   v_threshold = v_threshold)
cn.layer_converter(net_quan)

Quantized:  flat
Quantized:  linear1
Quantized:  lif1
Quantized:  linear2
Quantized:  lif2
Quantization time: 0.0010223388671875
Constructing axons from linear Layer
Input layer shape(infeature, outfeature): [ 1 28 28] 1000
Numer of neurons: 0, number of axons: 784
Constructing neurons from linear Layer
Hidden layer shape(infeature, outfeature):  (1000,) 10
Numer of neurons: 1000, number of axons: 784


### **Initiate the HiAER Spike SNN**

In [12]:
config = {}
config['neuron_type'] = "I&F"
config['global_neuron_params'] = {}
config['global_neuron_params']['v_thr'] = int(qn.v_threshold)
    
# #Have to be deployed on the FPGA
# hardwareNetwork = CRI_network(dict(cn.axon_dict),
#                               connections=dict(cn.neuron_dict),
#                               config=config,target='CRI', 
#                               outputs = cn.output_neurons,
#                               coreID=1)

softwareNetwork = CRI_network(dict(cn.axon_dict),
                              connections=dict(cn.neuron_dict),
                              config=config,target='simpleSim', 
                              outputs = cn.output_neurons,
                              coreID=1)

NameError: name 'neuron' is not defined

### **Deploying the SNN on HiAER Spike**

run_sw and run_hw are two helper functions for running the spiking neural network on HiAER-Spike.

In [ ]:
def Run_sw(self,inputList,softwareNetwork):
    predictions = []
    total_time_cri = 0
    #each image
    for currInput in tqdm(inputList):
        #reset the membrane potential to zero
        softwareNetwork.simpleSim.initialize_sim_vars(len(self.neuron_dict))
        spikeRate = [0]*10
        #each time step
        for slice in currInput:
            start_time = time.time()
            swSpike = softwareNetwork.step(slice, membranePotential=False)

            end_time = time.time()
            total_time_cri = total_time_cri + end_time-start_time
            for spike in swSpike:
                spikeIdx = int(spike) - self.bias_start_idx 
                try: 
                    if spikeIdx >= 0: 
                        spikeRate[spikeIdx] += 1 
                except:
                    print("SpikeIdx: ", spikeIdx,"\n SpikeRate:",spikeRate)
        predictions.append(spikeRate.index(max(spikeRate)))
    print(f"Total simulation execution time: {total_time_cri:.5f} s")
    return(predictions)

In [ ]:
def run_CRI_hw(self,inputList,hardwareNetwork):
    predictions = []
    #each image
    total_time_cri = 0
    for currInput in tqdm(inputList):
        #initiate the hardware for each image
        hs_bridge.FPGA_Execution.fpga_controller.clear(len(self.neuron_dict), False, 0)  ##Num_neurons, simDump, coreOverride
        spikeRate = [0]*10
        for slice in tqdm(currInput):
            start_time = time.time()
            hwSpike, latency, hbmAcc = hardwareNetwork.step(slice, membranePotential=False)
            # print(f'hwSpike: {hwSpike}\n. latency : {latency}\n. hbmAcc:{hbmAcc}')
            end_time = time.time()
            total_time_cri = total_time_cri + end_time-start_time
            for spike in hwSpike:
                # print(int(spike))
                spikeIdx = int(spike) - self.bias_start_idx 
                try: 
                    if spikeIdx >= 0: 
                        spikeRate[spikeIdx] += 1 
                except:
                    print("SpikeIdx: ", spikeIdx,"\n SpikeRate:",spikeRate)
        predictions.append(spikeRate.index(max(spikeRate))) 
    print(f"Total execution time CRIFPGA: {total_time_cri:.5f} s")
    return(predictions)

In [ ]:
cri_convert.bias_start_idx = int(cri_convert.output_neurons[0])
loss_fun = nn.MSELoss()
start_time = time.time()
test_loss = 0
test_acc = 0
test_samples = 0
num_batches = 0

RUN_HARDWARE = False #set to True if running on FPGA

for img, label in tqdm(test_loader):
    cri_input = cri_convert.input_converter(img)
    output = None
    if RUN_HARDWARE:
        output = torch.tensor(run_CRI_hw(cri_input,hardwareNetwork), dtype=float)
    else:
        output = torch.tensor(run_CRI_sw(cri_input,softwareNetwork), dtype=float)
    loss = loss_fun(output, label)
    test_samples += label.numel()
    test_loss += loss.item() * label.numel()
    test_acc += (output == label).float().sum().item()
    num_batches += 1
test_time = time.time()
test_speed = test_samples / (test_time - start_time)
test_loss /= test_samples
test_acc /= test_samples

print(f'test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}')
print(f'test speed ={test_speed: .4f} images/s')